In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

# Homology analysis by BLAST or Diamond

# Blastp

## DDX18

In [18]:
record_protein = SeqIO.read(open("CDS_prot_DDX18_seq.fasta"), format="fasta") 
print(len(record_protein.seq))

670


In [19]:
result_handle=NCBIWWW.qblast("blastp","swissprot",record_protein.seq)#blast with the protein translate of my cds
with open("blastp_file_ddx18.xml","w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

In [20]:
result_seq = open("blastp_file_ddx18.xml")
blast_results = NCBIXML.read(result_seq)
#print("Number of alignments:",len(blast_results.alignments))
for parameter in blast_results.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  Q9NVP1
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18; AltName: Full=Myc-regulated DEAD box protein; Short=MrDb [Homo sapiens]
E-value:  0.0

Accession:  Q8K363
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18 [Mus musculus]
E-value:  0.0

Accession:  Q9VD51
Definition:  RecName: Full=Probable ATP-dependent RNA helicase pitchoune [Drosophila melanogaster]
E-value:  0.0

Accession:  Q1EA54
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Coccidioides immitis RS]
E-value:  0.0

Accession:  A4R8B5
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Pyricularia oryzae 70-15]
E-value:  0.0


In [22]:
# results_Blastp= open('blastp_file_ddx18.xml')
# blastp_records = NCBIXML.read(results_Blastp)
# E_VALUE_THRESH = 0.001
# list_filtered_alignments,list_species=[],[]
# for alignment in  blastp_records.alignments:
#     for hsp in alignment.hsps:
# #         print(hsp.identities)    # maybe add more 
#         if hsp.expect < E_VALUE_THRESH:
#             list_filtered_alignments.append(alignment.accession)
#             title_organism=re.search(r'\[.+\s.+\]', alignment.title)
#             if title_organism:
#                 m = re.match(r'\[.+\s.+\]', title_organism[0] )
#                 specie = m.group(0)
#                 #print(specie)
#                 list_species.append(specie)
# for x in sorted(set(list_species)):
#     print("number of times: {} that appeared specie: {}".format(list_species.count(x),x))

# print(list_filtered_alignments)

In [24]:
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records = NCBIXML.read(results_Blastp)
E_VALUE_THRESH = 0.00001
list_filtered_alignments,list_species=[],[]
for alignment in  blastp_records.alignments:
    for hsp in alignment.hsps:
#         print(hsp.identities)    # maybe add more 
        if hsp.expect < E_VALUE_THRESH:
            list_filtered_alignments.append(alignment.accession)
            title_organism=re.search(r'\[.+\s.+\]', alignment.title)
            if title_organism:
                m = re.match(r'\[.+\s.+\]', title_organism[0] )
                specie = m.group(0)
                #print(specie)
                list_species.append(specie)
print('Number of times:    that appeared specie ')              
for x in sorted(set(list_species)):
        print("{:>8} - - - - {:<20}".format(list_species.count(x),x))

Number of times:    that appeared specie 
       2 - - - - [Arabidopsis thaliana]
       2 - - - - [Aspergillus clavatus NRRL 1]
       2 - - - - [Aspergillus fischeri NRRL 181]
       2 - - - - [Aspergillus fumigatus Af293]
       2 - - - - [Aspergillus nidulans FGSC A4]
       2 - - - - [Aspergillus niger CBS 513.88]
       2 - - - - [Aspergillus oryzae RIB40]
       2 - - - - [Aspergillus terreus NIH2624]
       1 - - - - [Candida albicans SC5314]
       2 - - - - [Chaetomium globosum CBS 148.51]
       2 - - - - [Coccidioides immitis RS]
       1 - - - - [Cryptococcus neoformans var. neoformans JEC21] >sp|P0CQ85.1| RecName: Full=ATP-dependent RNA helicase HAS1 [Cryptococcus neoformans var. neoformans B-3501A]
       1 - - - - [Debaryomyces hansenii CBS767]
       2 - - - - [Dictyostelium discoideum]
       1 - - - - [Drosophila melanogaster]
       1 - - - - [Eremothecium gossypii ATCC 10895]
       1 - - - - [Fusarium graminearum PH-1]
       1 - - - - [Gallus gallus]     
       

In [106]:
# with open('blastp_files_ID_ddx18.txt', 'w') as f:
#     for line in list_filtered_alignments:
#         f.write(f"{line}\n")

In [25]:
Blast=open("blastp_file_ddx18.xml")
Blast_record=NCBIXML.parse(Blast)
for x in Blast_record:
    print(x.database)
    print(x.matrix)
    print(x.gap_penalties)
    print(len(x.alignments))

swissprot
BLOSUM62
(11, 1)
50


In [26]:
result_handle=open("blastp_file_ddx18.xml")
blast_record=NCBIXML.read(result_handle)
E_VALUE_THRESH=0.0001
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect<E_VALUE_THRESH:
            print("----Aligment----")
            print("sequence:",alignment.title)
            print("lenght:",alignment.length)
            print("e.value:",hsp.expect) 
            print(hsp.query[0:75] + "...")
            print(hsp.query[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

----Aligment----
sequence: sp|Q9NVP1.2| RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18; AltName: Full=Myc-regulated DEAD box protein; Short=MrDb [Homo sapiens]
lenght: 670
e.value: 0.0
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
----Aligment----
sequence: sp|Q8K363.1| RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18 [Mus musculus]
lenght: 660
e.value: 0.0
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQ...
MSQLQMKLLRRKIEKRNAKLRQRNLKLQETSDTSLSQPQNGDVPKETGKGGKVKKALKRSVPVDSAEAQSGGMPE...
----Aligment----
sequence: sp|Q9VD51.2| RecName: Full=Probable ATP-dependent RNA helicase pitchoune [Drosophila melanogaster]
lenght: 680
e.valu

In [39]:
filtragem= []
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records_r = NCBIXML.read(results_Blastp)
first=blastp_records_r.alignments[0]
len_max_seq=first.hsps[0].align_length
print(len_max_seq)
results_Blastp= open('blastp_file_ddx18.xml')
blastp_records = NCBIXML.parse(results_Blastp)
list_accession,list_identities,list_coverage_start, list_coverage_stop,list_alignLen=[],[],[],[],[]
list_organism=[]
E_VALUE_THRESH = 0.00001
for alignment in  blastp_records:
    for a in alignment.alignments:
        for b in a.hsps:
            if b.expect < E_VALUE_THRESH:
                title_organism=re.search(r'\[.+\s.+\]', a.title)
                if title_organism:
                    m = re.match(r'\[(.+\s.+)\]', title_organism[0] )
                    list_accession.append(a.accession)
                    list_coverage_start.append(b.sbjct_start)
                    list_coverage_stop.append(b.sbjct_end)
                    list_identities.append(b.identities)
                    list_alignLen.append(b.align_length)
                    list_organism.append(m.group(1))
            #print(a.accession, b.align_length, b.identities, b.expect, b.sbjct_start, b.sbjct_end)
            
#calculation of %identity                        
def identity(list_i, list_a):
    perc_list_identities=[]
    for index in range(len(list_i)):
        perc_list_identities.append(round(list_i[index]*100/list_a[index]))
    return perc_list_identities

#calculation of the %coverage - for all hsps of all accessions
def coverage(list_end,list_start):
    perc_list_coverage=[]
    for index in range(len(list_identities)):
        perc_list_coverage.append(round((1+list_end[index]-list_start[index])/len_max_seq*100))
    return perc_list_coverage

#selecting the max coverage and max identity per hsps/accession
def max_values_by_id(ids, covs, idents): 
    unique_ids = []
    max_coverage = []
    max_identity = []
    for id, cov, ident in zip(ids, covs,idents):
        if id in unique_ids:
            idx = unique_ids.index(id)
            max_coverage[idx] = max(max_coverage[idx], cov)
            max_identity[idx] = max(max_identity[idx], ident)
        else:
            unique_ids.append(id)
            max_coverage.append(cov)
            max_identity.append(ident)
    return unique_ids, max_coverage, max_identity

beta_max=max_values_by_id(list_accession, coverage(list_coverage_stop, list_coverage_start),identity(list_identities,list_alignLen))

thresold_coverage=74
thresold_identity=59
print('__Thresholds__')
print('coverage: {}%   |'.format(thresold_coverage))
print('identity: {}%   |'.format(thresold_identity))
print('e-value< {} |\n'.format(E_VALUE_THRESH))
print('{:>5}{:>14}{:>11}{:>35}'.format('ID','%coverage','%identity', 'Organism'))
count=0
filtragem= []
for id, cov, ident, specie in zip(beta_max[0], beta_max[1], beta_max[2], list_organism):
    if int(cov)>thresold_coverage and ident>thresold_identity:
        if int(cov)>100:
            print('|{:>5}|{:>9}*{:>9}{:>24}'.format(id, cov, ident, specie))
            count+=1
        else:
            print('|{:>5}|{:>9}{:>10}{:>40}'.format(id, cov, ident, specie))
        filtragem.append(f"{id} {specie}")
  
if count>0:
    print('\n* means that the subject sequence is longer than the query sequence')

670
__Thresholds__
coverage: 74%   |
identity: 59%   |
e-value< 1e-05 |

   ID     %coverage  %identity                           Organism
|Q9NVP1|      100       100                            Homo sapiens
|Q8K363|       99        86                            Mus musculus
|A4R8B5|       78        60                Pyricularia oryzae 70-15
|A2Q9T6|       75        62            Aspergillus niger CBS 513.88
|A3LNR6|       75        61       Scheffersomyces stipitis CBS 6054
|Q7S2N9|       78        60                 Neurospora crassa OR74A


In [40]:
#teste
lista_sem_repetidos= []
seen = set()
for x in filtragem:
    parts = x.split()
    organi = parts[1] + '_' + parts[2]
    #print(parts)
    #print(organi)
    
    if organi in seen:
         filtragem.remove(x) 
    else:
        seen.add(organi)
        lista_sem_repetidos.append(x)      

lista_ids = []
with open('protein_result_blastp.txt', 'w') as f:
    for x in lista_sem_repetidos:
        print(x)
        IDS = x.split()
        lista_ids.append(IDS[0])
        f.write(f"{IDS[0]}\n")
    
# print(lista_ids)

Q9NVP1 Homo sapiens
Q8K363 Mus musculus
A4R8B5 Pyricularia oryzae 70-15
A2Q9T6 Aspergillus niger CBS 513.88
A3LNR6 Scheffersomyces stipitis CBS 6054
Q7S2N9 Neurospora crassa OR74A


In [111]:
# blast_qresult=SearchIO.read("blastp_file_ddx18.xml","blast-xml")
# blast_hsp=blast_qresult[0][0]
# print(blast_qresult[0][0])#best hsp
# print(blast_hsp.aln)